# Tutorial: QPU Database

**This tutorial requires version >=0.0.5 of the QPU DB**

## Using the QPU DB

The QPU database is a permanent store built for storing calibration data for Quantum Processing Units (QPU).

It provides the following features and benefits:

* Persistent storage of any python object related to QPU calibration info
* Metadata on parameter calibration state and last modified time
* Convenient addressing of quantum elements
* Easy revert to previously stored parameters

In this short tutorial we will learn how to use the QPU DB by looking at a simplified example of a QPU with two superconducting
qubits, two readout resonators and a parametric coupling element.

### Creating the database

Below we can see a simple usage example. The DB is created by calling the `create_new_database` method.
This method is similar to initializing a git repo in the sense that we only do it once. Here we initialize it
with an initial dictionary which contains some basic attributes of our QPU. We'll be able to add more attributes,
and also elements, later on. Once we call `create_new_qpu_database`, a set of database files will be created for us at
the working directory of the python script.

These files are the persistent storage of our DB. They can be saved to a different location by specifying
the `path` argument to the function.

In [1]:
# %load_ext autoreload
# %autoreload 2
from entropylab_qpudb import create_new_qpu_database, CalState, QpuDatabaseConnection

In [2]:
initial_dict = {
    'q1': {
        'f01': 5.65e9  # an initial guess for our transition frequency
    },
    'q2': {
        'f01': 5.25e9
    },
    'res1': {
        'f_r': 7.1e9
    },
    'res2': {
        'f_r': 7.3e9
    },
    'c1_2': {
        'f_r': 0.4e9
    }
}
create_new_qpu_database('db1', initial_dict, force_create=True)

Notes:

1. here we allow for the possibility of overwriting an existing database
by passing the `force_create=True` flag. This option is useful when experimenting with the database creation, however in
common usage it is recommended to remove this flag, since when it's false (by default), it will prevent overwriting an existing
database and deleting all the data stored in it.

2. (For experts): if you need to create a DB server, rather than create a filesystem storage, please let us know.
The DB backend is currently
the [ZODB](https://zodb.org/en/latest/) database, with plans to be replaced by
[gitdb](https://github.com/gitpython-developers/gitdb).

The keys of `initial_dict` are called the *elements* (and are similar in nature to QUA's quantum elements), and the
values of these elements are subdictionaries of *attributes*. The values of the attributes can be anything you like,
or more accurately, any python object that can be pickled. The different elements need not have the same attributes.

### Connecting to the database and basic usage

Now create a connection to our DB. The connection to the DB is our main "workhorse" - we create the DB once, and
whenever we want to connect to it in order to retrieve or store data, we open a connection object. Note that currently
only a single connection object per DB is allowed.

In [3]:
db1 = QpuDatabaseConnection('db1')

opening qpu database db1 from commit <timestamp: 05/30/2021 06:24:19, message: initial commit> at index 0


and let's view the contents of our DB by calling `print`:

In [11]:
db1.print()


q1
----
f01:	QpuParameter(value=5400000000.0, last updated: 05/30/2021 09:24:45, calibration state: COARSE)

q2
----
f01:	QpuParameter(value=5250000000.0, last updated: 05/30/2021 09:24:19, calibration state: UNCAL)

res1
----
f_r:	QpuParameter(value=7100000000.0, last updated: 05/30/2021 09:24:19, calibration state: UNCAL)

res2
----
f_r:	QpuParameter(value=7300000000.0, last updated: 05/30/2021 09:24:19, calibration state: UNCAL)

c1_2
----
f_r:	QpuParameter(value=400000000.0, last updated: 05/30/2021 09:24:19, calibration state: UNCAL)


Congratulations! You've just created your first QPU DB. As you can see when calling `print` the values we entered
in `initial_dict` are now objects of type `QpuParameter`. These objects have 3 attributes:

* `value`: the value you created initially and can be any python object
* `last_updated`: the time when this parameter was last updated (see *committing* section to understand how to
update). This parameter is handled by the DB itself.
* `cal_state`: an enumerated metadata that can take the values `UNCAL`, `COARSE`, `MED` and `FINE`. This
can be used by the user to communicate what is the calibration level of these parameters. They can be set and queried
during the script execution, but are not used by the DB itself.

### Modifying and using QPU parameters

We can use and modify values and calibration states of QPU parameters in two different ways:

#### Using `get` and `set`

let's modify the value of `f01` and then get the actual value:

In [5]:
db1.set('q1', 'f01', 5.33e9)
db1.get('q1', 'f01').value

5330000000.0

We can also modify the calibration state when setting:

In [6]:
db1.set('q1', 'f01', 5.36e9, CalState.COARSE)

To get the full `QpuParameter` object we can omit `.value`. We can see that the cal state and modification date were updated.

In [7]:
db1.get('q1', 'f01')
#db1.get('q1', 'f01').cal_state

QpuParameter(value=5360000000.0, last updated: 05/30/2021 09:24:35, calibration state: COARSE)

Note that we can't modify the value by assigning to value directly - this will raise an exception.

#### Using resolved names

The names we chose for the elements, namely `'q1'`, `'res1'` and `'c1_2'` have a special significance. If we follow this
convention of naming qubit elements with the format 'q'+number, resonators with the format 'res'+number
and couplers with the format 'c'+number1+'_'+number2, as shown above, this allows us to get and set values in a more
convenient way:

In [8]:
print(db1.q(1).f01.value)
print(db1.res(1).f_r.value)
print(db1.coupler(1, 2).f_r.value)
print(db1.coupler(2, 1).f_r.value)

5360000000.0
7100000000.0
400000000.0
400000000.0


while this method basically syntactic sugar, it allows us to conveniently address elements by indices, which is useful when
working with multiple qubit systems, and especially with couplers. We can also set values using this resolved addressing method:

In [9]:
db1.update_q(1, 'f01', 5.4e9)
db1.q(1).f01

QpuParameter(value=5400000000.0, last updated: 05/30/2021 09:24:45, calibration state: COARSE)

Note: This default mapping between integer indices and strings can be modified by subclassing the
`Resolver` class found under `entropylab_qpudb._resolver.py`.

### Committing (saving to persistent storage) and viewing history

Everything we've done so far did not modify the persistent storage. In order to do this, we need to *commit* the changes we made.
This allows us to control at which stages we want to make aggregated changes to the database.

Let's see how this is done. We need to call `commit`, and specify an optional commit message:

In [12]:
db1.update_q(1, 'f01', 6.e9)
db1.commit('a test commit')


commiting qpu database db1 with commit <timestamp: 05/30/2021 06:26:20, message: a test commit> at index 1


Now the actual file was changed. To see this, we need to close the db. We can then delete db1,
and when re-opening the DB we'll see f01 of q1 has the modified value.

In [26]:
db1.close()
del db1
db1 = QpuDatabaseConnection('db1')
db1.q(1).f01

closing qpu database db1
closing qpu database db1
opening qpu database db1 from commit <timestamp: 05/27/2021 06:44:34, message: a test commit> at index 1


QpuParameter(value=6000000000.0, last updated: 05/27/2021 09:44:34, calibration state: COARSE)

closing qpu database db1
closing qpu database db1
opening qpu database db1 from commit <timestamp: 05/27/2021 06:44:34, message: a test commit> at index 1


QpuParameter(value=6000000000.0, last updated: 05/27/2021 09:44:34, calibration state: COARSE)

Note that the commit was saved with an index. This index can be later used to revert to a [previous state](#reverting-to-a-previous-state).

To view a history of all the commits, we call `get_history`.

Note that the timestamps of the commits are in UTC time.

In [13]:
db1.get_history()

,timestamp,message
0,2021-05-30 06:24:19.796403,initial commit
1,2021-05-30 06:26:20.205781,a test commit


### Adding attributes and elements

In many cases you realize while calibrating your system that you want to add attributes that did not exist in the initial
dictionary, or even new elements. This is easy using the `add_element` and `add_attribute` methods.
Let's see an example for `add_attribute`:

In [14]:
db1.add_attribute('q1', 'anharmonicity')

print(db1.q(1).anharmonicity)

db1.update_q(1, 'anharmonicity', -300e6, new_cal_state=CalState.COARSE)

print(db1.q(1).anharmonicity)


QpuParameter(None)
QpuParameter(value=-300000000.0, last updated: 05/30/2021 09:26:25, calibration state: COARSE)


### Reverting to a previous state

Many times when we work on bringing up a QPU, we reach a point where everything is calibrated properly and our measurements
and calibrations give good results. We want to be able to make additional changes, but to possibly revert to the good state
if things go wrong. We can do this using `restore_from_history`. We simply need to provide it with the history
index to which we want to return:

In [15]:
db1.restore_from_history(0)
print(db1.q(1).f01)
assert db1.q(1).f01.value == initial_dict['q1']['f01']


opening qpu database db1 from commit <timestamp: 05/30/2021 06:24:19, message: initial commit> at index 0
QpuParameter(value=5650000000.0, last updated: 05/30/2021 09:24:19, calibration state: UNCAL)


Calling this method will replace the current working DB with the DB that was stored in the commit with the index
supplied to `restore_from_history`. The new values will not be committed. It is possible to modify the values and
commit them as usual.

## Next steps

While the QPU DB is a standalone tool, it is designed with QUA calibration node framework in mind.
In the notebook called `2_qubit_graph_calibration.ipynb` we explore how the QUA calibration nodes framework can be used
to generate calibration graphs.

## Remove DB files

To remove the DB files created in your workspace for the purpose of this demonstration, first close the db connection:

In [19]:
db1.close()

closing qpu database db1


then run this cell:

In [20]:
from glob import glob
import os
for fl in glob("db1*"):
    os.remove(fl)